In [1]:
import tensorflow as tf
from dataset.dataflows import get_dataflow

In [2]:
def gen(df):
    def f():
        for i in df:
            yield tuple(i[0])
            yield tuple(i[1])

    return f

In [7]:
annot_path_val = 'd:///coco-dataset/annotations/person_keypoints_val2017.json'
img_dir_val = 'd://coco-dataset/val2017/'
batch_size = 5
strict = True
x_size = 224
y_size = 28

In [8]:
df, size = get_dataflow(
    annot_path=annot_path_val,
    img_dir=img_dir_val,
    strict=strict,
    x_size=x_size,
    y_size=y_size,
    lmdb_file="lmdb_file"
)
df.reset_state()

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
Loading image annot 0/5000
Loading image annot 2000/5000
Loading image annot 4000/5000
[DEBUG] has prepared 4396 coco-data meta
[DEBUG] read img ok
[DEBUG] augment ok
[DEBUG] build sample ok


In [ ]:
def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [ ]:
# with tf.io.TFRecordWriter("./val.tfrecord") as f:
#     for dp in df:
#         en_input_feature = list(
#             map(lambda seq_input:, seq)
#              )


In [9]:
ds = tf.data.Dataset.from_generator(
    gen(df), (tf.float32, tf.float32, tf.float32),
    output_shapes=(
        tf.TensorShape([x_size, x_size, 3]),
        tf.TensorShape([y_size, y_size, 38]),
        tf.TensorShape([y_size, y_size, 19])
    )
)

In [13]:
print(ds)
print(ds.element_spec)
for i, _ in enumerate(ds.element_spec):
    ds_i = ds.map(lambda *args: args[i]).map(tf.io.serialize_tensor)
    print(ds_i)
    writer = tf.data.experimental.TFRecordWriter(f'mydata.{i}.tfrecord')
    writer.write(ds_i)

<FlatMapDataset shapes: ((224, 224, 3), (28, 28, 38), (28, 28, 19)), types: (tf.float32, tf.float32, tf.float32)>


In [21]:
num_parts = 3
data = []
def read_map_fn(x):
    return tf.io.parse_tensor(x,tf.float32)
for i in range(num_parts):
    data.append(tf.data.TFRecordDataset(f"mydata.{i}.tfrecord").map(read_map_fn))
    rds = tf.data.Dataset.zip(tuple(data))

print(data)
print(len(rds))
for x in rds:
    print(x)
    break

[<MapDataset shapes: <unknown>, types: tf.float32>, <MapDataset shapes: <unknown>, types: tf.float32>, <MapDataset shapes: <unknown>, types: tf.float32>]


TypeError: dataset length is unknown.

In [ ]:
def write_map_fn(x1,x2,x3):
    return tf.io.serialize_tensor(tf.concat[x1,x2,x3],-1)

In [15]:
from tensorflow.io import serialize_tensor
ds = ds.map(lambda x0,x1,x2:(serialize_tensor(x0),serialize_tensor(x1),serialize_tensor(x2)))
# w0 = tf.data.experimental.TFRecordWriter("./val0.tfrecord")
# w1 = tf.data.experimental.TFRecordWriter("./val1.tfrecord")
# w2 = tf.data.experimental.TFRecordWriter("./val2.tfrecord")
# wl = [w0,w1,w2]

In [ ]:
import os
print(os.getcwd())

In [ ]:
for idx,x in enumerate(ds):
    print(idx)
    print(x[0].dtype)
    tf.print(x[0].numpy())
    # w0.write(x[0].type)
    # w1.write(x[1])
    # w2.write(x[2])
    break
# w0.write(ds)

In [16]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def se(img,paf,hp):
    feature = {
        'img' : _bytes_feature(img),
        'paf' : _bytes_feature(paf),
        'hp' : _bytes_feature(hp)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializePartialToString()

In [18]:
# from t

filename = './val.tfrecord'
# writer = tf.data.experimental.TFRecordWriter(filename)
writer = tf.io.TFRecordWriter(filename)
# writer.write()
for x in tqdm(ds):
    # if ix % 100 == 0:
    writer.write(se(x[0],x[1],x[2]))

34it [05:45, 10.16s/it]


KeyboardInterrupt: 